In [ ]:
import pandas as pd
import numpy as np

# Load dataset
file_path = "/content/dataset.csv"
df_actual = pd.read_csv(file_path)
# Step 1: Combine date and time into a single timestamp for sorting
df_actual['Timestamp'] = pd.to_datetime(df_actual['LastUpdatedDate'] + ' ' + df_actual['LastUpdatedTime'],
                                        format='%d-%m-%Y %H:%M:%S')

# Step 2: Encode traffic condition (assumed mapping)
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
df_actual['TrafficLevel'] = df_actual['TrafficConditionNearby'].map(traffic_map)

# Step 3: Encode vehicle type with weights
vehicle_weights = {'bike': 0.5, 'car': 1.0, 'truck': 1.5}
df_actual['VehicleWeight'] = df_actual['VehicleType'].map(vehicle_weights)

# Step 4: Sort data per lot by timestamp
df_actual = df_actual.sort_values(by=['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

# Initialize base price
base_price = 10.0

# ---- Model 1: Linear Pricing with Occupancy ----
def model_1_real(df, alpha=2):
    df = df.copy()
    df['Price_Model1'] = base_price

    for i in range(1, len(df)):
        if df.loc[i, 'SystemCodeNumber'] == df.loc[i - 1, 'SystemCodeNumber']:
            occ_ratio = df.loc[i, 'Occupancy'] / df.loc[i, 'Capacity']
            df.loc[i, 'Price_Model1'] = max(base_price, df.loc[i - 1, 'Price_Model1'] + alpha * occ_ratio)
    return df

# ---- Model 2: Demand-Based Pricing ----
def model_2_real(df, alpha=1, beta=0.5, gamma=0.3, delta=2, epsilon=1, lam=0.2):
    df = df.copy()
    # Compute demand score
    df['DemandScore'] = (alpha * (df['Occupancy'] / df['Capacity']) +
                         beta * df['QueueLength'] -
                         gamma * df['TrafficLevel'] +
                         delta * df['IsSpecialDay'] +
                         epsilon * df['VehicleWeight'])

    # Normalize demand
    min_demand = df['DemandScore'].min()
    max_demand = df['DemandScore'].max()
    df['NormDemand'] = (df['DemandScore'] - min_demand) / (max_demand - min_demand)

    # Price calculation
    df['Price_Model2'] = base_price * (1 + lam * df['NormDemand'])
    df['Price_Model2'] = df['Price_Model2'].clip(lower=0.5 * base_price, upper=2 * base_price)
    return df

# ---- Model 3: Competitive Pricing Based on Nearby Lots ----
def model_3_real(df, lot_coords, competitor_prices, distance_threshold=0.001):
    df = df.copy()
    df['Price_Model3'] = df['Price_Model2']

    for i, row in df.iterrows():
        lot_id = row['SystemCodeNumber']
        lat, lon = row['Latitude'], row['Longitude']
        timestamp = row['Timestamp']
        own_price = row['Price_Model3']
        is_full = row['Occupancy'] >= row['Capacity']

        # Find nearby lots
        nearby_prices = []
        for comp_lot, (clat, clon) in lot_coords.items():
            if comp_lot == lot_id:
                continue
            dist = np.sqrt((lat - clat)**2 + (lon - clon)**2)
            if dist < distance_threshold:
                price = competitor_prices.get((comp_lot, timestamp), base_price)
                nearby_prices.append(price)

        if nearby_prices:
            avg_comp_price = np.mean(nearby_prices)
            if is_full and own_price > avg_comp_price:
                df.at[i, 'Price_Model3'] = max(base_price, avg_comp_price * 0.95)
            elif not is_full and own_price < avg_comp_price:
                df.at[i, 'Price_Model3'] = min(2 * base_price, own_price * 1.05)
    return df

# Sample locations for parking lots
lot_coords = {
    lot: (float(coords['Latitude']), float(coords['Longitude']))
    for lot, coords in df_actual.groupby('SystemCodeNumber')[['Latitude', 'Longitude']].first().to_dict('index').items()
}

# Generate sample competitor prices
timestamps = df_actual['Timestamp'].unique()
competitor_prices = {(lot, ts): np.random.uniform(8, 12) for lot in lot_coords.keys() for ts in timestamps}

# Run all models
df_model1 = model_1_real(df_actual)
df_model2 = model_2_real(df_model1)
df_model3 = model_3_real(df_model2, lot_coords, competitor_prices)

# Display final output
cols_to_show = ['SystemCodeNumber', 'Timestamp', 'Occupancy', 'QueueLength', 'TrafficLevel',
                'IsSpecialDay', 'VehicleWeight', 'Price_Model1', 'Price_Model2', 'Price_Model3']
df_model3[cols_to_show].head()



,SystemCodeNumber,Timestamp,Occupancy,QueueLength,TrafficLevel,IsSpecialDay,VehicleWeight,Price_Model1,Price_Model2,Price_Model3
0,BHMBCCMKT01,2016-10-04 07:59:00,61,1,1.0,0,1.0,10.000000,10.211619,10.722200
1,BHMBCCMKT01,2016-10-04 08:25:00,64,1,1.0,0,1.0,10.221837,10.212631,10.723263
2,BHMBCCMKT01,2016-10-04 08:59:00,80,2,1.0,0,1.0,10.499133,10.315403,10.831173
3,BHMBCCMKT01,2016-10-04 09:32:00,107,2,1.0,0,1.0,10.870017,10.324516,10.840741
4,BHMBCCMKT01,2016-10-04 09:59:00,150,2,1.0,0,0.5,11.389948,10.241657,10.241657


In [ ]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# Upload your dataset
file_path = "/content/dataset.csv"
df_actual = pd.read_csv(file_path)

# Load data
df = pd.read_csv(file_path)

# Combine date and time into a single timestamp
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                 format='%d-%m-%Y %H:%M:%S')

# Encode traffic level and vehicle weights
traffic_map = {'low': 1, 'medium': 2, 'high': 3}
vehicle_weights = {'bike': 0.5, 'car': 1.0, 'truck': 1.5}

df['TrafficLevel'] = df['TrafficConditionNearby'].map(traffic_map)
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights).fillna(0.3)  # for unknowns like 'cycle'

# Sort chronologically per lot
df = df.sort_values(by=['SystemCodeNumber', 'Timestamp']).reset_index(drop=True)

# Step 1: Compute base demand-based price
df['OccupancyRatio'] = df['Occupancy'] / df['Capacity']
df['RawDemand'] = (
    1.0 * df['OccupancyRatio'] +
    0.3 * df['QueueLength'] -
    0.2 * df['TrafficLevel'] +
    0.5 * df['IsSpecialDay'] +
    0.4 * df['VehicleWeight']
)

df['NormalizedDemand'] = df.groupby('SystemCodeNumber')['RawDemand'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min() + 1e-6)
)

df['BasePrice'] = 10 * (1 + 0.7 * df['NormalizedDemand'])
df['BasePrice'] = df['BasePrice'].clip(lower=5, upper=20).round(4)

# Step 2: Calculate location info of each lot
lot_locations = df.groupby("SystemCodeNumber")[["Latitude", "Longitude"]].first().to_dict('index')

# Step 3: Compute average competitor price (within 0.5 km and ±15 min)
def compute_avg_competitor_price(row, radius_km=0.5):
    current_lot = row['SystemCodeNumber']
    current_time = row['Timestamp']
    current_coord = (row['Latitude'], row['Longitude'])

    nearby_prices = []

    for lot, coords in lot_locations.items():
        if lot == current_lot:
            continue
        dist = geodesic(current_coord, (coords['Latitude'], coords['Longitude'])).km
        if dist <= radius_km:
            # Allow ±15 minutes window
            time_mask = (
                (df['SystemCodeNumber'] == lot) &
                (abs((df['Timestamp'] - current_time).dt.total_seconds()) <= 900)
            )
            price_series = df.loc[time_mask, 'BasePrice']
            if not price_series.empty:
                nearby_prices.append(price_series.mean())

    if nearby_prices:
        return np.mean(nearby_prices)
    else:
        return np.nan  # will be handled later

# Apply competitor logic
df['AvgCompetitorPrice'] = df.apply(compute_avg_competitor_price, axis=1)

# Step 4: Final competitive pricing adjustment
df['BasePrice'] = df['BasePrice'].fillna(method='ffill')
df['AvgCompetitorPrice'] = df['AvgCompetitorPrice'].fillna(df['BasePrice'])

theta = 0.4  # sensitivity to competitor pricing
df['FinalPrice'] = df['BasePrice'] + theta * (df['BasePrice'] - df['AvgCompetitorPrice'])
df['FinalPrice'] = df['FinalPrice'].clip(lower=5, upper=20).round(2)

# Step 5: Preview output
df[['SystemCodeNumber', 'Timestamp', 'Occupancy', 'QueueLength',
    'BasePrice', 'AvgCompetitorPrice', 'FinalPrice']].head(15)


/tmp/ipython-input-2-2396916013.py:77: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['BasePrice'] = df['BasePrice'].fillna(method='ffill')


,SystemCodeNumber,Timestamp,Occupancy,QueueLength,BasePrice,AvgCompetitorPrice,FinalPrice
0,BHMBCCMKT01,2016-10-04 07:59:00,61,1,10.9933,11.1894,10.91
1,BHMBCCMKT01,2016-10-04 08:25:00,64,1,11.0021,11.0021,11.00
2,BHMBCCMKT01,2016-10-04 08:59:00,80,2,11.5569,11.3342,11.65
3,BHMBCCMKT01,2016-10-04 09:32:00,107,2,11.6361,11.0886,11.86
4,BHMBCCMKT01,2016-10-04 09:59:00,150,2,11.4237,11.6694,11.33
5,BHMBCCMKT01,2016-10-04 10:26:00,177,3,12.3495,12.2656,12.38
6,BHMBCCMKT01,2016-10-04 10:59:00,219,6,13.6579,13.6579,13.66
7,BHMBCCMKT01,2016-10-04 11:25:00,247,5,13.6579,13.6579,13.66
8,BHMBCCMKT01,2016-10-04 11:59:00,259,5,13.6579,13.6579,13.66
9,BHMBCCMKT01,2016-10-04 12:29:00,266,8,14.1344,14.3167,14.06
